In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### 파일 압축해제 (1회만 하면 됨)

In [ ]:
!mkdir labels
%cd labels
!unzip -qq "/content/drive/MyDrive/한국어_방언_발화(경상도)/[라벨]경상도_학습데이터_1.zip"

/content/labels


In [ ]:
%cd ..
!mkdir ksd_4
%cd ksd_4
!unzip -qq "/content/drive/MyDrive/한국어_방언_발화(경상도)/[원천]경상도_4.zip"

/content
/content/ksd_4


In [ ]:
%cd ..
!mkdir ksd_7
%cd ksd_7
!unzip -qq "/content/drive/MyDrive/한국어_방언_발화(경상도)/[원천]경상도_7.zip"

/content
/content/ksd_7


### 제반 작업

In [ ]:
!pip install librosa

In [ ]:
import os
import librosa
import soundfile as sf
import math
import re
import json

### 방언 정제 함수 정의

In [ ]:
# 방언 텍스트 파일 정제
def rule(x):
    # 괄호
    a = re.compile(r'\([^)]*\)')
    # 문장 부호
    b = re.compile('[^가-힣 ]')
    x = re.sub(pattern=a, repl='', string= x)
    x = re.sub(pattern=b, repl='', string= x)
    return x

# 예제 문장
# x = "&company-name2&요거."

# # 처리 결과 출력
# result = rule(x)
# print(result)

### 폴더 생성 및 파일 분리

In [ ]:
#각자 사정에 맞게 모든 파일이 있는 폴더 경로(절대경로)를 입력해줍니다.
label_dir = '/content/labels/'
audio_dir = ['/content/ksd_4/', '/content/ksd_7/']
destination = '/content/drive/MyDrive/한국어_방언_발화(경상도)/Training/train_extracted/' #최종 파일이 저장될 폴더명

In [ ]:
"""
프로세스:
1. json 열기
2. json파일명과 동일한 폴더 새로 생성
3. 해당 wav 열기
4. 폴더 안에 하위 id별로 잘린 wav파일과 dialect form txt파일 저장
"""

for dir in audio_dir:
    for file in sorted(os.listdir(dir)):

        #파일명으로 된 폴더 생성하는 과정, 없으면 pass를 통해 오류 방지
        filename = file.split('.')[0]
        # print(destination+filename)
        try:
            os.mkdir(destination+filename)
            print(f'folder made successfully: {destination+filename}')
        except:
            if len(os.listdir(destination+filename))==0 or len(os.listdir(destination+filename)) % 2 != 0:
                print(f'folder is empty or error, try generating: {destination+filename}')
                pass
            else:
                print(f'folder already exists, pass: {destination+filename}')
                continue

        #json 파일 파싱
        with open(label_dir + filename + '.json') as f:
            data = json.load(f)

            #오디오 미리 불러오기
            y, sr = librosa.load(dir+filename+'.wav', sr=16000)

            for x in data['utterance']:
                id, dialect_txt, start, end = x["id"], x["dialect_form"], x["start"], x["end"]
                id = id.replace(".", "_") #혹시 파일명 오류날까봐 .를 _로 대체하기

                # audio trimming -> int형태로만 자를 수 있어, 시작 초 내림~끝 초 올림 으로 분할
                ny = y[math.floor(start)*sr : sr*math.ceil(end)]
                sf.write(destination+filename+"/"+id+".wav", ny, sr)

                dialect_txt = rule(dialect_txt)
                #텍스트 파일 저장
                with open(destination+filename+"/"+id+".txt", 'w') as txt:
                    txt.write(dialect_txt)


folder already exists, pass: /content/drive/MyDrive/한국어_방언_발화(경상도)/Training/train_extracted/DKSR20002316
folder already exists, pass: /content/drive/MyDrive/한국어_방언_발화(경상도)/Training/train_extracted/DKSR20002317
folder already exists, pass: /content/drive/MyDrive/한국어_방언_발화(경상도)/Training/train_extracted/DKSR20002318
folder already exists, pass: /content/drive/MyDrive/한국어_방언_발화(경상도)/Training/train_extracted/DKSR20002319
folder already exists, pass: /content/drive/MyDrive/한국어_방언_발화(경상도)/Training/train_extracted/DKSR20002320
folder already exists, pass: /content/drive/MyDrive/한국어_방언_발화(경상도)/Training/train_extracted/DKSR20002321
folder already exists, pass: /content/drive/MyDrive/한국어_방언_발화(경상도)/Training/train_extracted/DKSR20002322
folder already exists, pass: /content/drive/MyDrive/한국어_방언_발화(경상도)/Training/train_extracted/DKSR20002323
folder already exists, pass: /content/drive/MyDrive/한국어_방언_발화(경상도)/Training/train_extracted/DKSR20002324
folder already exists, pass: /content/drive/MyDrive/한국어

In [ ]:
drive.flush_and_unmount()

KeyboardInterrupt: 